# Transformer (Translation)

In [1]:
import random
import numpy as np
import tensorflow as tf
from konlpy.tag import Okt

from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Lambda, Layer, Embedding, LayerNormalization

In [12]:
#%pwd

In [3]:
EPOCHS = 200
NUM_WORDS = 2000

## Dot-Scaled Attention

In [7]:
class DotScaledAttention(Layer):
    def __init__(self, d_emb, d_reduced, masked=False):
        super().__init__()
        self.q = Dense(d_reduces, input_shape = (-1,d_emb))
        self.k = Dense(d_reduces, input_shape = (-1,d_emb))
        self.v = Dense(d_reduces, input_shape = (-1,d_emb))
        self.scale = Lambda(lambda x: x/np.sqrt(d_reduced)) #d_k
        self.masked = masked

    def call(self, x, training=None, mask=None): # x : (q,k,v)
        q = self.scale(self.q(x[0]))
        k = self.k(x[1])
        v = self.v(x[2])
        k_T = tf.transpose(k, perms = [0,2,1]) # 1번째는 그대로 두고 2,3번째를 transpose
        comp = tf.matmul(q, k_T)

        if self.masked: # Referred from https://github.com/LastRemote/Transformer-TF2.0
            length = tf.shape(comp)[-1]
            mask = tf.fill((length, length), -np.inf) #길이만큼(현재 시점의 이전만큼) 참조하고 뒷부분은 -무한대
            mask = tf.linalg.band_part(mask, 0, -1) # Get upper triangle
            mask = tf.linalg.set_diag(mask, tf.zeros((length))) # Set diagonal to zeros to avoid operations with infinity
            comp += mask #만약 -무한대가 생기면 softmax할때 0으로 수렴.
            
        comp = tf.nn.softmax(comp, axis = -1)
        return tf.matmul(comp, v)

## Multi-Head Attention

In [6]:
class MultiHeadAttention(Layer):
    def __init__(self, num_head, d_emb, d_reduced, masked=False):
        super().__init__()
        self.attention_list = []
        for _ in range(num_head):
            self.attention_list.append(DotScaledAttention(d_emb, d_reduced, masked))
        self.linear = Dense(d_emb, input_shape = (-1, num_head * d_reduced))

    def call(self, x, training=None, mask=None):
        attention_list = [a(x) for a in self.attention_list]
        concat = tf.concat(attention_list, axis = -1)
        
        return self.linear(concat)

## Encoder

In [8]:
class Encoder(Layer):
    def __init__(self, num_head, d_reduced):
        super().__init__()
        self.num_head = num_head
        self.d_r = d_reduced
    
    def build(self, input_shape):
        self.multi_attention = MultiHeadAttention(self.num_head, input_shape[-1], self.d_r)
        self.layer_norm1 = LayerNormalization(input_shape = input_shape)
        self.dense1 = Dense(input_shape[-1] * 4, input_shape = input_shape, activation = 'relu')
        self.dense2 = Dense(input_shape[-1], input_shape = self.dense1.compute_output_shape(input_shape), activation = 'relu')
        self.layer_norm2 = LayerNormalization(input_shape = input_shape)
        super().build(input_shape)

    def call(self, x, training=None, mask=None):
        h = self.multi_attention((x, x, x))
        ln1 = self.layer_norm1(x + h)
        
        h = self.dense2(self.dense1(ln1)) #feed forward
        return self.layer_norm2(h + ln1)

    def compute_output_shape(self, input_shape):
        return input_shape

## Decoder

In [9]:
class Decoder(Layer):
    def __init__(self, num_head, d_reduced):
        super().__init__()
        self.num_head = num_head
        self.d_r = d_reduced
    
    def build(self, input_shape):
        self.self_attention = MultiHeadAttention(self.num_head, input_shape[0][-1], self.d_r, masked = True)
        self.layer_norm1 = LayerNormalization(input_shape = input_shape)
        self.multi_attention = MultiHeadAttention(self.num_head, input_shape[0][-1], self.d_r)
        self.layer_norm2 = LayerNormalization(input_shape = input_shape)
        self.dense1 = Dense(input_shape[0][-1] * 4, input_shape = input_shape[0], activation = 'relu')
        self.dense2 = Dense(input_shape[0][-1], input_shape = self.dense1.compute_output_shape(input_shape[0]), activation = 'relu')
        self.layer_norm3 = LayerNormalization(input_shape = input_shape)
        super().build(input_shape)

    def call(self, inputs, training=None, mask=None): #inputs : (x, context from encoder)
        x, context = inputs
        h = self.self_attention((x,x,x))
        ln1 = self.layer_norm1(x + h)
        
        h = self.multi_attention((ln1, context, context))
        ln2 = self.layer_norm2(ln1 + h)
        
        h = self.dense2(self.dense1(ln2))
        return self.layer_norm3(ln2 + h)

    def compute_output_shape(self, input_shape):
        return input_shape

## Positional Encoding

In [10]:
class PositionalEncoding(Layer):  # Referred from https://github.com/LastRemote/Transformer-TF2.0
    def __init__(self, max_len, d_emb):
        super().__init__()
        self.sinusoidal_encoding = np.array([self.get_positional_angle(pos, d_emb) for pos in range(max_len)], dtype=np.float32)
        self.sinusoidal_encoding[:, 0::2] = np.sin(self.sinusoidal_encoding[:, 0::2])
        self.sinusoidal_encoding[:, 1::2] = np.cos(self.sinusoidal_encoding[:, 1::2])
        self.sinusoidal_encoding = tf.cast(self.sinusoidal_encoding, dtype=tf.float32)

    def call(self, x, training=None, mask=None):
        return x + self.sinusoidal_encoding[:tf.shape(x)[1]]

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_angle(self, pos, dim, d_emb):
        return pos / np.power(10000, 2 * (dim // 2) / d_emb)

    def get_positional_angle(self, pos, d_emb):
        return [self.get_angle(pos, dim, d_emb) for dim in range(d_emb)]

## Transformer Architecture

In [11]:
class Transformer(Model):
    def __init__(self, src_vocab, dst_vocab, max_len, d_emb, d_reduced, n_enc_layer, n_dec_layer, num_head):
        super().__init__()
        self.enc_emb = Embedding(src_vocab, d_emb)
        self.dec_emb = Embedding(dst_vocab, d_emb)
        self.pos_enc = PositionalEncoding(max_len, d_emb)
        
        self.encoder = [Encoder(num_head, d_reduced) for _ in range(n_enc_layer)]
        self.decoder = [Decoder(num_head, d_reduced) for _ in range(n_dec_layer)]
        self.dense = Dense(dst_vocab, input_shape = (-1, d_emb))

    def call(self, inputs, training=None, mask=None): #input : (src_sentence, shifted_dst_sentence)
        src_s, shifted_dst_s = inputs
        
        h_enc = self.pos_enc(self.enc_emb(src_s))
        for enc in self.encoder:
            h_enc = enc(h_enc)
            
        h_dec = self.pos_enc(self.dec_emb(shifted_dst_s))
        for dec in self.decoder:
            h_dec = dec([h_dec, h_enc])
            
        return tf.nn.softmax(self.dense(h_dec), axis = -1)

## Dataset 준비

In [10]:
dataset_file = 'chatbot_data.csv' # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)
y_train_shifted = np.concatenate([np.zeros((y_train.shape[0], 1)), y_train[:, 1:]], axis=1)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

## Train using keras

In [ ]:
transformer = Transformer(NUM_WORDS, NUM_WORDS, 128, 16, 16, 2, 2, 4) # Instantiating a new transformer model

transformer.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
transformer.fit([x_train, y_train_shifted], y_train, batch_size=5, epochs=EPOCHS)

Train on 399 samples
Epoch 1/200
399/399 [==============================] - 9s 22ms/sample - loss: 5.9527 - accuracy: 0.8479
Epoch 2/200
399/399 [==============================] - 1s 4ms/sample - loss: 2.7419 - accuracy: 0.8900
Epoch 3/200
399/399 [==============================] - 2s 4ms/sample - loss: 1.0273 - accuracy: 0.8900
Epoch 4/200
399/399 [==============================] - 1s 4ms/sample - loss: 0.7265 - accuracy: 0.8900
Epoch 5/200
399/399 [==============================] - 1s 3ms/sample - loss: 0.6082 - accuracy: 0.8991
Epoch 6/200
399/399 [==============================] - 1s 3ms/sample - loss: 0.5347 - accuracy: 0.9207
Epoch 7/200
399/399 [==============================] - 1s 3ms/sample - loss: 0.4885 - accuracy: 0.9238
Epoch 8/200
399/399 [==============================] - 1s 3ms/sample - loss: 0.4585 - accuracy: 0.9265
Epoch 9/200
399/399 [==============================] - 1s 3ms/sample - loss: 0.4344 - accuracy: 0.9265
Epoch 10/200
399/399 [=============================